### Dataset Loader


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/degrees.zip

Archive:  /content/drive/MyDrive/degrees.zip
   creating: degrees/
   creating: degrees/large/
  inflating: degrees/large/stars.csv  
  inflating: degrees/large/people.csv  
  inflating: degrees/large/movies.csv  
  inflating: degrees/util.py         
  inflating: degrees/degrees.py      
   creating: degrees/small/
  inflating: degrees/small/stars.csv  
  inflating: degrees/small/people.csv  
  inflating: degrees/small/movies.csv  


In [4]:
import pandas as pd

In [5]:
movies=pd.read_csv('/content/degrees/small/movies.csv')
people=pd.read_csv('/content/degrees/small/people.csv')
stars=pd.read_csv('/content/degrees/small/stars.csv')

In [15]:
movies.head()

,id,title,year
0,112384,Apollo 13,1995
1,104257,A Few Good Men,1992
2,109830,Forrest Gump,1994
3,93779,The Princess Bride,1987
4,95953,Rain Man,1988


In [16]:
people.head()

,id,name,birth
0,102,Kevin Bacon,1958
1,129,Tom Cruise,1962
2,144,Cary Elwes,1962
3,158,Tom Hanks,1956
4,1597,Mandy Patinkin,1952


In [9]:
stars

,person_id,movie_id
0,102,104257
1,102,112384
2,129,104257
3,129,95953
4,144,93779
5,158,109830
6,158,112384
7,1597,93779
8,163,95953
9,1697,93779


### Utils

In [10]:
%%writefile /content/degrees/util.py 

class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action


class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node


class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node




Overwriting /content/degrees/util.py


### a) Degress of Seperation: BFS

In [11]:
%%writefile /content/degrees/degrees_bfs.py

import csv
import sys

from util import StackFrontier, QueueFrontier
from util import  Node

# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}


def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"/content/degrees/small/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"/content/degrees/small/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"/content/degrees/small/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


def main():
    if len(sys.argv) > 2:
        sys.exit("Usage: python degrees.py [directory]")
    directory = sys.argv[1] if len(sys.argv) == 2 else "large"

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    path = shortest_path(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")


def shortest_path(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.
    Returns the shortest path from source to target using Depth First Search.
    If no possible path, returns None.
    """
    solution_found = False
    no_solution = False
    solution = list()

    initial = Node(state=source, parent=None, action=None)
    frontier = QueueFrontier()
    frontier.add(initial)
    explored = set()
    i = 0
    while solution_found == False:

        if frontier.empty() == True:
            no_solution = True
            solution_found = True
        
        node = frontier.remove()
        # print("\n\nNode in= ",node, ' i=', i)

        if node.state == target:
            # Return the solution
            solution_found = True
            while node.parent is not None:
                pid, mid = node.state, node.action
                solution.append((mid, pid))
                node = node.parent
            solution.reverse()

        explored.add(node)
        children = neighbors_for_person(node.state)
        for child in children:
            child_node = Node(state=child[1], action=child[0],parent=node)
            frontier.add(child_node)
            if child_node.state == target:
                # Return the solution
                solution_found = True
                while child_node.parent is not None:
                    pid, mid = child_node.state, child_node.action
                    solution.append((mid, pid))
                    child_node = child_node.parent
                solution.reverse()
    
    if solution_found == True:
        if no_solution == True:
            return None
        return solution

def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors


if __name__ == "__main__":
    main()

Writing /content/degrees/degrees_bfs.py


### As seen in the shortest path function, BFS uses a "Queue" to traverse a graph. Breadth First Search explores the shallowest nodes.
Breadth First Search, finds the shortest path first. To implement Breadth First Search, we use a queue data structure to store nodes, and the algorithm works by adding the neighbors of a node to the queue and exploring them before moving on to the next level.

Here is the code that implements Breadth First Search to find the shortest path between two actors:

In [23]:
!python /content/degrees/degrees_bfs.py

Loading data...
Data loaded.
Name: Mandy Patinkin
Name: Chris Sarandon
1 degrees of separation.
1: Mandy Patinkin and Chris Sarandon starred in The Princess Bride


### b) Degress of Seperation: DFS

In [21]:
%%writefile /content/degrees/degrees_dfs.py

import csv
import sys

from util import StackFrontier, QueueFrontier
from util import  Node

# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}


def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"/content/degrees/small/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"/content/degrees/small/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"/content/degrees/small/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


def main():
    if len(sys.argv) > 2:
        sys.exit("Usage: python degrees.py [directory]")
    directory = sys.argv[1] if len(sys.argv) == 2 else "large"

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    path = shortest_path(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")

def shortest_path(source, target):
    """
    Returns the shortest path from source to target using Depth First Search.
    """
    start = Node(state=source, parent=None, action=None)
    frontier = StackFrontier()
    frontier.add(start)
    explored = set()

    while True:
        if frontier.empty():
            raise Exception("No solution found")

        node = frontier.remove()
        explored.add(node.state)

        for movie, actor in neighbors_for_person(node.state):
            if actor not in explored and not frontier.contains_state(actor):
                child = Node(state=actor, parent=node, action=movie)

                if child.state == target:
                    path = []
                    while child.parent is not None:
                        path.append((child.action, child.state))
                        child = child.parent
                    path.reverse()
                    return path

                frontier.add(child)

def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors


if __name__ == "__main__":
    main()

Writing /content/degrees/degrees_dfs.py


In [24]:
!python /content/degrees/degrees_bfs.py

Loading data...
Data loaded.
Name: Dustin Hoffman
Name: Valeria Golino
1 degrees of separation.
1: Dustin Hoffman and Valeria Golino starred in Rain Man
